In [20]:
#############
# LIBRARIES #
#############

import os
import pandas as pd
import numpy as np

import streamlit as st

import pickle as pkl

import warnings
warnings.filterwarnings("ignore")

#########################################
# LOADING MODEL AND ENCODER/TRANSFORMER #
#########################################

loaded_extraTree_model = pkl.load(open('../Model/final_model.pickle', 'rb'))
loaded_encoder = pkl.load(open('../Model/feature_encoder.pickle', 'rb'))
OldPeak_tranformer = pkl.load(open('../Model/OldPeak_tranformer.pickle', 'rb'))
Cholesterol_transformer = pkl.load(open('../Model/Cholesterol_transformer.pickle', 'rb'))

"""
Data dictionary:
- Age: age of the patient (years)
- Sex: sex of the patient (M: Male, F: Female)
- ChestPainType: chest pain type (TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic)
- RestingBP: resting blood pressure (mm Hg)
- Cholesterol: serum cholesterol (mm/dl)
- FastingBS: fasting blood sugar (1: if FastingBS > 120 mg/dl, 0: otherwise)
- RestingECG: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]
- MaxHR: maximum heart rate achieved (Numeric value between 60 and 202)
- ExerciseAngina: exercise-induced angina (Y: Yes, N: No)
- Oldpeak: oldpeak = ST (Numeric value measured in depression)
- ST_Slope: the slope of the peak exercise ST segment (Up: upsloping, Flat: flat, Down: downsloping)
- HeartDisease: output class [1: heart disease, 0: Normal]
"""

########################################################
# FUNCTIONS FOR PROCESSING DATA AND MAKING PREDICTIONS #
########################################################

def process_data(user_input_df):
    # user_input_df = pd.DataFrame(user_input) # Creating a dataframe containing user inputs

    cat_cols = user_input_df.select_dtypes(include='object').columns
    user_input_df[cat_cols] = user_input_df[cat_cols].astype('category')

    user_input_df['FastingBS'] = np.where(user_input_df['FastingBS']=='No', 0, 1)
    user_input_df['FastingBS'] = user_input_df['FastingBS'].astype('category') # Converting to type category for transforming
    
    # Encoding categorical features
    categorical_columns = user_input_df.select_dtypes('category').columns
    user_input_df[categorical_columns] = loaded_encoder.transform(user_input_df[categorical_columns])

    # Transforming data
    transformed_OldPeak = OldPeak_tranformer.transform(user_input_df[['Oldpeak']])
    transformed_Cholesterol = Cholesterol_transformer.transform(user_input_df[['Cholesterol']])
    transformed_RestingBP = np.log(user_input_df[['RestingBP']])
    user_input_df['Oldpeak'] = transformed_OldPeak
    user_input_df['Cholesterol'] = transformed_Cholesterol
    user_input_df['RestingBP'] = transformed_RestingBP

    return user_input_df

def make_prediction(user_input_df):
    prediction = loaded_extraTree_model.predict(user_input_df)
    prediction_probability = loaded_extraTree_model.predict_proba(user_input_df).max()

    results = prediction, prediction_probability
    return results

In [21]:
# User inputs
age = int(input('Enter your age: '))
sex = input('Enter your sex (M or F): ')
restingBP = int(input('Enter your resting blood pressure: '))
cholesterol = int(input('Enter your cholesterol level: '))
fastingBS = input('Is you blood sugar level above 120 dl/mg when fasting (Y or N): ')
MaxHR = int(input('Enter your max heart rate: '))
ExerciseAngina = input('Do you experience excercise-enduced engina (Y or N): ')
Oldpeak = int(input('Enter your old peak value: '))
ChestPainType = input('Describe you type of chest pains (ATA, NAP, ASY, TA): ')
stSlope = input('Enter your ST Slode (Up, Flat, Down): ')
RestingECG = input('Enter your resting ECG (Normal, ST, LVH): ')

In [22]:
user_input = {
                'Age': [age], 
                'Sex': [sex],
                'ChestPainType': [ChestPainType], 
                'RestingBP': [restingBP], 
                'Cholesterol': [cholesterol],
                'FastingBS': [fastingBS],
                'RestingECG': [RestingECG],
                'MaxHR': [MaxHR],
                'ExerciseAngina': [ExerciseAngina],
                'Oldpeak': [Oldpeak],
                'ST_Slope': [stSlope],
}
user_input_df = pd.DataFrame(user_input) # Creating a dataframe containing user inputs

user_input_df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,41,M,ATA,140,180,Y,ST,190,Y,1,Flat


In [23]:
processed_data = process_data(user_input_df)
results = make_prediction(processed_data)

In [24]:
if results[0] == 0:
    print(f'You are not at risk for heart disease.--Prediction Confidence: {"{:.0%}".format(results[1])}.')
else:
    print(f'You ARE at risk for heart disease!--Prediction Confidence: {"{:.0%}".format(results[1])}.')

You ARE at risk for heart disease!--Prediction Confidence: 81%.


In [12]:
#############
# STREAMLIT #
#############

st.set_page_config(page_title="Healthy Heart App",page_icon="⚕️",layout="centered",initial_sidebar_state="expanded")

